In [19]:
import pandas as pd
import numpy as np

In [20]:
# Load CSV file (betting spreads)
df = pd.read_csv("nba_data/nba_betting_spread.csv")

# Display the first few rows
df.head()

# Show all column names
print(df.columns)

# Filter for BetOnline
df_betOnline = df[df['book_name'] == 'BetOnline']

# Ensure only one entry per Game ID for BetOnline
print(df_betOnline['game_id'].value_counts())

# The Filter Dataset
df_betOnline.to_csv('nba_data/nba_betting_spread_betOnline.csv', index=False)

# Verify the Filtered Data
print(df_betOnline.head())
print(df_betOnline.shape)

Index(['game_id', 'book_name', 'book_id', 'team_id', 'a_team_id', 'spread1',
       'spread2', 'price1', 'price2'],
      dtype='object')
game_id
21000358    1
20901228    1
40900121    1
40900131    1
40900161    1
           ..
21700693    1
21700694    1
21700696    1
21700698    1
21501153    1
Name: count, Length: 13789, dtype: int64
     game_id  book_name  book_id     team_id   a_team_id  spread1  spread2  \
3   21000358  BetOnline     1096  1610612749  1610612742      7.5     -7.5   
12  21000361  BetOnline     1096  1610612755  1610612751     -1.5      1.5   
21  21000362  BetOnline     1096  1610612747  1610612764    -10.5     10.5   
30  21000367  BetOnline     1096  1610612747  1610612754     -3.5      3.5   
39  21000372  BetOnline     1096  1610612766  1610612763      8.5     -8.5   

    price1  price2  
3   -110.0  -110.0  
12  -110.0  -110.0  
21  -110.0  -110.0  
30  -110.0  -110.0  
39  -110.0  -110.0  
(13789, 9)


In [21]:
df = pd.read_csv("nba_data/nba_betting_spread_betOnline.csv")

# List of columns to drop (replace with actual columns you don't need)
columns_to_drop2 = ["book_name", "book_id", "price1", "price2"]

# Remove the unnecessary columns
df_cleaned = df.drop(columns=columns_to_drop2)

# Save the cleaned dataset
df_cleaned.to_csv("nba_data/nba_betting_spread_cleaned.csv", index=False)

print("Unnecessary columns removed, and dataset saved as nba_betting_spread_cleaned.csv!")

Unnecessary columns removed, and dataset saved as nba_betting_spread_cleaned.csv!


In [22]:
# Load the dataset
df = pd.read_csv("nba_data/nba_games_all.csv")  # Use your preprocessed file

# Filter out rows where season_year == 2006
df_filtered = df[df["season_year"] != 2006]

# Save the cleaned dataset with a new name
df_filtered.to_csv("nba_data/nba_games_all_filtered.csv", index=False)

print("Dataset with season 2006 removed saved as nba_games_filtered_season_2006_removed.csv!")

Dataset with season 2006 removed saved as nba_games_filtered_season_2006_removed.csv!


In [23]:
# Load the cleaned dataset
df = pd.read_csv("nba_data/nba_games_all_filtered.csv")

# List of columns to drop (replace with actual columns you don't need)
columns_to_drop = ["game_date", "matchup", "w", "l", "season_year", "min", "fgm", "fga", "fg_pct", "fg3m", "fg3a", "fg3_pct", "ftm", "fta", "ft_pct", "oreb", "dreb", "reb", "ast", "stl", "blk", "tov", "pf", "pts", "season"]

# Remove the unnecessary columns
df_cleaned = df.drop(columns=columns_to_drop)

# Save the cleaned dataset
df_cleaned.to_csv("nba_data/nba_games_final.csv", index=False)

print("Unnecessary columns removed, and dataset saved as nba_games_final_cleaned.csv!")

Unnecessary columns removed, and dataset saved as nba_games_final_cleaned.csv!


In [24]:
# Load the core dataset
games_df = pd.read_csv("nba_data/nba_games_final.csv")

# Load the betting spread dataset
spreads_df = pd.read_csv("nba_data/nba_betting_spread_cleaned.csv")

# Merge the datasets on game_id
merged_df = games_df.merge(spreads_df[["game_id", "spread1", "spread2"]], on="game_id", how="left")

# Save the final dataset
merged_df.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Final dataset created! Saved as factors_and_spreads.csv")

Final dataset created! Saved as factors_and_spreads.csv


In [25]:
# Load the dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")  # Your dataset

# Filter the dataset to include only games where spread1 > 0 (i.e., favorites)
df_favorites = df[df["spread1"] > 0]

# Drop duplicates based on game_id, keeping only the first occurrence
df_favorites_unique = df_favorites.drop_duplicates(subset=["game_id"], keep="first")

# Save the cleaned dataset
df_favorites_unique.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Filtered dataset saved as factors_and_spreads_filtered.csv! Only games with spread1 > 0 are kept and duplicates are removed.")

Filtered dataset saved as factors_and_spreads_filtered.csv! Only games with spread1 > 0 are kept and duplicates are removed.


In [26]:
# Load the dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Remove rows where 'w_pct' is NaN
df_cleaned = df.dropna(subset=["w_pct"])

# Save the cleaned dataset back to CSV
df_cleaned.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Rows with missing 'w_pct' have been removed and the dataset is saved.")

Rows with missing 'w_pct' have been removed and the dataset is saved.


In [27]:
# Load dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Create is_upset column: True if the team was an underdog and won, OR if the favorite lost
df["is_upset"] = ((df["spread1"] > 0) & (df["wl"] == "W"))

# Save the updated dataset
df.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Updated dataset saved as factors_and_spreads.csv with 'is_upset' column!")

Updated dataset saved as factors_and_spreads.csv with 'is_upset' column!


In [28]:
# Load the dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Filter out Preseason games
df = df[df["season_type"] != "Pre Season"]

# Save the cleaned dataset
df.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Preseason games removed. Dataset updated and saved!")

Preseason games removed. Dataset updated and saved!


In [29]:
# Load dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Check if Playoff games have different w_pct patterns
print(df.groupby("season_type")["w_pct"].describe())  

# Step 1: Calculate each team's regular-season win percentage
regular_season_wpct = (
    df[df["season_type"] == "Regular Season"]
    .groupby("team_id")["w_pct"]
    .last()  # Get the last available w_pct (final reg season record)
)

# Step 2: Update Playoff games' w_pct to their team's regular-season w_pct
df.loc[df["season_type"] == "Playoffs", "w_pct"] = df["team_id"].map(regular_season_wpct)

# Save the updated dataset
df.to_csv("nba_data/factors_and_spreads.csv", index=False)

print("Fixed playoff w_pct! Saved updated dataset.")


                 count      mean       std  min    25%    50%    75%  max
season_type                                                              
Playoffs         720.0  0.515860  0.305558  0.0  0.333  0.571  0.685  1.0
Regular Season  8504.0  0.498654  0.174712  0.0  0.375  0.500  0.617  1.0
Fixed playoff w_pct! Saved updated dataset.


In [30]:
# Load dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Check the overall percentage of upsets
upset_rate = df["is_upset"].mean()
print(f"Overall Upset Rate: {upset_rate:.2%}")

Overall Upset Rate: 49.10%


In [31]:
homecourt_upset_rate = df.groupby("is_home")["is_upset"].mean()
print(homecourt_upset_rate)

is_home
f    0.282035
t    0.707910
Name: is_upset, dtype: float64


In [32]:
season_type_upset_rate = df.groupby("season_type")["is_upset"].mean()
print(season_type_upset_rate)

season_type
Playoffs          0.475000
Regular Season    0.492357
Name: is_upset, dtype: float64


In [33]:
# Load the cleaned dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Check unique values in is_home
print(df['is_home'].unique())

# Map 't' to True, 'f' to False, and handle NaN values
df['is_home'] = df['is_home'].map({'t': True, 'f': False})
df['is_home'] = df['is_home'].fillna(False)  # If there are any NaN values, treat them as False

# Verify that mapping worked
print(df['is_home'].unique())  # Should return [True, False]

# Group by 'is_home' and calculate upsets and games
upset_summary = df.groupby('is_home').agg(
    upsets=('is_upset', 'sum'),  # Count total upsets
    games=('is_upset', 'count')  # Count total games
)

# Calculate proportion of upsets
upset_summary['prop'] = upset_summary['upsets'] / upset_summary['games']

# Print the summary table
print(upset_summary)

print('An underdog team playing away won 28.2% of the time')
print('An underdog team playing at home won 70.8% of the time')

['f' 't']
[False  True]
         upsets  games      prop
is_home                         
False      1325   4698  0.282035
True       3204   4526  0.707910
An underdog team playing away won 28.2% of the time
An underdog team playing at home won 70.8% of the time


In [34]:
# Bin teams into four groups based on win percentage
df["w_pct_bin"] = pd.qcut(df["w_pct"], q=4, labels=["Low", "Mid-Low", "Mid-High", "High"])

# Group by w_pct_bin and calculate upset rates
w_pct_upset_summary = df.groupby("w_pct_bin").agg(
    upsets=("is_upset", "sum"),  # Total upsets
    games=("is_upset", "count")  # Total games
)

# Compute upset proportion
w_pct_upset_summary["prop"] = w_pct_upset_summary["upsets"] / w_pct_upset_summary["games"]

# Print the results
print(w_pct_upset_summary)
print('Teams with a win percentage in the bottom 25th percentile win as an underdog 26.7% of the time.')
print('Teams with a win percentage in the 25-50th percentile win as an underdog 45.1% of the time.')
print('Teams with a win percentage in the 50-75th percentile win as an underdog 55.8% of the time.')
print('Teams with a win percentage in the top 25th percentile win as an underdog 69.1% of the time.')

           upsets  games      prop
w_pct_bin                         
Low           627   2345  0.267377
Mid-Low      1024   2272  0.450704
Mid-High     1289   2308  0.558492
High         1589   2299  0.691170
Teams with a win percentage in the bottom 25th percentile win as an underdog 26.7% of the time.
Teams with a win percentage in the 25-50th percentile win as an underdog 45.1% of the time.
Teams with a win percentage in the 50-75th percentile win as an underdog 55.8% of the time.
Teams with a win percentage in the top 25th percentile win as an underdog 69.1% of the time.


/var/folders/2t/0j9q5vjs2_l2pp_7wflhtr3w0000gn/T/ipykernel_58975/701655469.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  w_pct_upset_summary = df.groupby("w_pct_bin").agg(


In [39]:
# Load the cleaned dataset
df = pd.read_csv("nba_data/factors_and_spreads.csv")

# Step 1: Map 't' to True, 'f' to False, and handle NaN values for is_home
df['is_home'] = df['is_home'].map({'t': True, 'f': False})
df['is_home'] = df['is_home'].fillna(False)  # If there are any NaN values, treat them as False

# Step 2: Bin teams into four groups based on win percentage
df["w_pct_bin"] = pd.qcut(df["w_pct"], q=4, labels=["Low", "Mid-Low", "Mid-High", "High"])

# Step 3: Create a new column that combines 'is_home' and 'w_pct_bin' for easy grouping
df['home_w_pct_bin'] = df['is_home'].map({True: 'Home', False: 'Away'}) + " " + df['w_pct_bin'].astype(str)

# Step 4: Group by the new 'home_w_pct_bin' column and calculate upset statistics
upset_summary = df.groupby('home_w_pct_bin').agg(
    upsets=('is_upset', 'sum'),  # Total upsets
    games=('is_upset', 'count')  # Total games
)

# Step 5: Compute upset proportion
upset_summary['prop'] = upset_summary['upsets'] / upset_summary['games']

# Step 6: Reset index for better readability
upset_summary = upset_summary.reset_index()

# Print the summary table
print(upset_summary)

# Interpret results
for _, row in upset_summary.iterrows():
    print(f"{row['home_w_pct_bin']}: {row['prop']*100:.2f}% upset rate")


  home_w_pct_bin  upsets  games      prop
0      Away High     297    705  0.421277
1       Away Low     315   1728  0.182292
2  Away Mid-High     331    986  0.335700
3   Away Mid-Low     382   1279  0.298671
4      Home High    1292   1594  0.810540
5       Home Low     312    617  0.505673
6  Home Mid-High     958   1322  0.724660
7   Home Mid-Low     642    993  0.646526
Away High: 42.13% upset rate
Away Low: 18.23% upset rate
Away Mid-High: 33.57% upset rate
Away Mid-Low: 29.87% upset rate
Home High: 81.05% upset rate
Home Low: 50.57% upset rate
Home Mid-High: 72.47% upset rate
Home Mid-Low: 64.65% upset rate
